In [ ]:
# !pip install azure-storage-blob
# !pip install python-dotenv

import os
from dotenv import load_dotenv
from setup_utils import fetch_data, load_data, create_time_index
import pandas as pd
import numpy as np

CONNECTION_STRING = os.getenv("CONNECTION_STRING")

load_dotenv()
fetch_data(CONNECTION_STRING)

(
    brand_mapping,
    macro_data,
    brand_constraint,
    pack_constraint,
    segment_constraint,
    sales_data,
    volume_variation_constraint,
) = load_data()


(
    macro_data,
    sales_data,
) = create_time_index([macro_data, sales_data])


In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

In [ ]:
# target_sku_list = sales_data[sales_data.gto.isna()].sku.unique()
# target_brand_list = sales_data[sales_data.gto.isna()][["sku", "brand"]].brand.unique()

# k = sales_data[["sku", "brand"]].drop_duplicates().groupby("sku").apply(lambda x: set(x.brand.unique()))#[target_sku_list]
# k2 = sales_data[["sku", "brand"]].drop_duplicates().groupby("sku").apply(lambda x: x.brand.nunique()>1)

# issue_skus = k[k2].drop_duplicates().to_list()

# brand_date_range = sales_data.fillna(0).reset_index().groupby("brand").apply(lambda x: pd.Series([x.date.min(), x.date.max()], index=["min", "max"]))
# brand_date_range = brand_date_range.to_dict()

In [ ]:
df = sales_data.reset_index().groupby(["date"])[["volume", "net_revenue", "promotional_discount", "other_discounts"]].sum().sort_index().join(macro_data)
k = df.net_revenue.shift(1)
k.name = "shifted_nr"
df = df.join(k).fillna(method="bfill")